In [1]:
import pandas as pd
import numpy as np
import datetime

# Functions for precursors calculation

In [2]:
def running_avg(sonde_file, parameters, windowsize=15, count_err=False):
    """
      Функция читает переданный в sonde_file файл с показаниями ионозонда
    и производит подсчет скользящего среднего характеристик из списка parameters
    с окном размера windowsize. Опция count_err добавляет подсчет среднего
    значения ошибки показаний (если показания были восстановлены интерполяцией).
    """
    t = pd.read_csv(sonde_file, sep='\t')
    err_features = []
    if count_err:
        err_features = list(filter(lambda x: x.endswith('_err'), t.columns.values))
    t = t[['sonde', 'year', 'date', 'h', 'm'] + parameters + err_features]
    t = t.sort_values(by=['date', 'h', 'm'])
    
    res = pd.DataFrame()
    
    for h, m in t[['h', 'm']].drop_duplicates().values: 
        t_subset = t.loc[(t.h == h) & (t.m == m)]
        ra = t_subset[parameters + err_features].rolling(15, min_periods=1).mean().shift(1)
        nd = t_subset[parameters].rolling(15, min_periods=1).count().shift(1)
        
        ra.rename(index=int,
                  columns=dict(zip(parameters + err_features,
                                   [p + '_running_avg' for p in (parameters + err_features)])),
                  inplace=True)
        nd.rename(index=int,
                  columns=dict(zip(parameters, [p + '_n_days' for p in parameters])),
                  inplace=True)
        
        res = pd.concat([res, t_subset.join(ra).join(nd)])
        
    return res

In [3]:
def sonde_subset(sonde, parameters, dates, range_):
    subset = pd.DataFrame()
    
    sonde['date'] = sonde['date']\
                   .apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
        
    if range_ == 'all':
        return sonde[['sonde', 'year', 'date', 'h', 'm'] + parameters]
        
    min_date = sonde['date'].min()
    sonde['day_temp'] = sonde['date'].apply(lambda x: (x - min_date).days)
    
    for date in dates:
        day = (date - min_date).days
        subset = pd.concat([subset,
                           sonde.loc[(sonde.day_temp - day >= range_[0]) 
                                    & (sonde.day_temp - day <= range_[1])][
                               ['sonde', 'year', 'date', 'h', 'm'] + parameters]
                           ])
    return subset


def correlation(sonde_file1, sonde_file2, parameters, earthquakes_dates, range_=[-10, 4]):
    """
      Функция считает дневную корреляцию характеристик из списка
    parameters для зондов, показания которых записаны в файлах
    sonde_file1 и sonde_file2. Корреляция считается для временных 
    интервалов (d + range_[0], d + range_[1]) для каждой даты d из
    earthquakes_dates, если range_ задан двумя числами, или для 
    всех возможных дней, если range_=='all'.
    """
    s1 = pd.read_csv(sonde_file1, sep='\t')
    s2 = pd.read_csv(sonde_file2, sep='\t')
    
    earthquakes_dates = [datetime.datetime.strptime(d, '%Y-%m-%d') 
                         for d in earthquakes_dates]
    
    s1 = sonde_subset(s1, parameters, earthquakes_dates, range_)
    s2 = sonde_subset(s2, parameters, earthquakes_dates, range_)

    s1.rename(index=int,
             columns=dict(zip(['sonde'] + parameters, 
                              ['sonde1'] + [p + '1' for p in parameters])),
             inplace=True)
    s2.rename(index=int,
             columns=dict(zip(['sonde'] + parameters, 
                              ['sonde2'] + [p + '2' for p in parameters])),
             inplace=True)
    
    merged = pd.merge(s1, s2, how='inner', on=['year', 'date', 'h', 'm'])
    res = pd.DataFrame()
    row = {
            'sonde1': merged.sonde1.values[0],
            'sonde2': merged.sonde2.values[0],
        }
    
    for d, y in merged[['date', 'year']].drop_duplicates().values:
        row['date'] = d
        row['year'] = int(y)
        subset = merged.loc[merged.date == d]
        for p in parameters:
            row[p + '_corr'] = subset[p+'1'].corr(subset[p+'2'], min_periods=2)
            row[p + '_n_hours'] = subset[['h', p+'1', p+'2']].dropna().h.nunique()
            row[p + '_total_time_points'] = subset[['h', p+'1', p+'2']].dropna().shape[0]
            
        res = res.append(row, ignore_index=True)
    
    return res

In [4]:
def groups_comparison(eq, in_, out_, parameters):
    """
      Функция для указанного в eq землетрясения 
    создает таблицу, состоящую из показателей parameters
    для группы зондов внутри зоны подготовки землетрясения
    (из таблицы in_) и снаружи (из таблицы out_). 
    """
    s1, s1_sondes = merge_sondes(in_, eq, in_, parameters)
    s2, s2_sondes = merge_sondes(out_, eq, in_, parameters)
    date = in_[in_.earthquake_id == eq].date.values[0] 
    
    if len(s1_sondes) < 2 or len(s2_sondes) < 2:
        return pd.DataFrame()
    
    s1 = label_group(s1, date, 's1', parameters)
    s2 = label_group(s2, date, 's2', parameters)
    
    ds = pd.merge(s1, s2, how='inner', on=['date', 'h', 'm'])
    ds['lbl'] = ds.apply(lambda x: 'pre_eq' if x['lbl_x'] == 'pre_eq' or x['lbl_y'] == 'pre_eq' \
                        else 'not_eq', axis=1)
    ds.drop(columns=['lbl_x', 'lbl_y'], inplace=True)
    ds['deviation'] = (ds.foF2_avg_s1 - ds.foF2_avg_s2).abs()
    return ds

def merge_sondes(ds, eq, in_, parameters):
    s = pd.DataFrame()
    s_sondes = []

    for sonde in ds[ds.earthquake_id==eq].sonde.values:
        t = pd.read_csv(f'../NCEI_dataset/ionosondes_data_corrected/{sonde}_corrected.csv',
                               sep='\t', parse_dates=['date'])
        s_sondes.append(sonde)
        
        t = t[['date', 'h', 'm'] + parameters]
        t.rename(columns=dict(zip(parameters, 
                                 [p+sonde for p in parameters])), 
                 inplace=True)
        
        t = remove_pre_eq_days(t, in_[(in_.sonde == sonde) 
                                      & (in_.earthquake_id != eq)].date.values)
        
        if s.shape[0] == 0:
            s = t.copy()
        else:
            s = pd.merge(s, t, how='outer', on=['date', 'h', 'm'])

    for p in parameters:
        columns = list(filter(lambda x: x.startswith(p), s.columns.values))
        s[p+'_avg'] = s[columns].apply(np.nanmean, axis=1)
        s[p+'_n_sondes'] = (~s[columns].isnull()).sum(axis=1)

    return s, s_sondes

def remove_pre_eq_days(df, dates):
    if len(dates) == 0:
        return df
    min_date = min(dates)
    dates_num = [(d - min_date).astype('timedelta64[D]') / np.timedelta64(1, 'D')
                 for d in dates]
    df['temp_day_numb'] = df.date.apply(lambda x: (x - min_date).days)
    
    for n in dates_num:
        df = df[df.temp_day_numb.apply(lambda x: x < n - 10 or x > n - 4)]
        
    return df.drop(columns=['temp_day_numb'])

def label_group(ds, date, name, parameters):
    ds['lbl'] = 'not_eq'
    ds.loc[ds.date.apply(lambda x: ((x - date).days >= -7) and ((x - date).days <= 0)),
          'lbl'] = 'pre_eq'

    for p in parameters:
        ds.rename(columns={p+'_avg': p+'_avg_'+name,
                           p+'_n_sondes': p+'_n_sondes_'+name}, 
                     inplace=True)
    
    return ds

# Example of using

In [5]:
vt139 = running_avg('../NCEI_dataset/ionosondes_data_corrected/VT139_corrected.csv', ['foF2'],
                   count_err=True)

In [6]:
vt139.head()

,sonde,year,date,h,m,foF2,foF2_err,foF2_running_avg,foF2_err_running_avg,foF2_n_days
0,VT139,1999,1999-06-28,0,0,7.75,0.0,NaN,NaN,NaN
96,VT139,1999,1999-06-29,0,0,7.45,0.0,7.75,0.0,1.0
192,VT139,1999,1999-06-30,0,0,8.35,0.0,7.60,0.0,2.0
288,VT139,1999,1999-07-01,0,0,8.85,0.0,7.85,0.0,3.0
384,VT139,1999,1999-07-02,0,0,9.35,0.0,8.10,0.0,4.0


In [7]:
vt_vs_ra = correlation('../NCEI_dataset/ionosondes_data/VT139.csv', 
                       '../NCEI_dataset/ionosondes_data/RA041.csv',
                      ['foF2'], ['2019-09-21'])

In [8]:
vt_vs_ra.head(10)

,date,foF2_corr,foF2_n_hours,foF2_total_time_points,sonde1,sonde2,year
0,2019-09-11,0.960367,24.0,80.0,VT139,RA041,2019.0
1,2019-09-12,0.968764,23.0,87.0,VT139,RA041,2019.0
2,2019-09-13,0.936932,22.0,68.0,VT139,RA041,2019.0
3,2019-09-14,0.960135,21.0,74.0,VT139,RA041,2019.0
4,2019-09-15,0.936880,23.0,76.0,VT139,RA041,2019.0
5,2019-09-16,0.920758,22.0,67.0,VT139,RA041,2019.0
6,2019-09-17,0.942272,24.0,91.0,VT139,RA041,2019.0
7,2019-09-18,0.974411,23.0,80.0,VT139,RA041,2019.0
8,2019-09-19,0.978056,20.0,66.0,VT139,RA041,2019.0
9,2019-09-20,0.949523,22.0,80.0,VT139,RA041,2019.0


In [6]:
in_ = pd.read_csv('../USGS_dataset/sondes_in_eq_prep_zone.csv', sep='\t',
                 parse_dates=['date'])
out_ = pd.read_csv('../USGS_dataset/sondes_out_eq_prep_zone.csv', sep='\t',
                 parse_dates=['date'])
s1s2 = groups_comparison('usp000h2gd', in_, out_, ['foF2'])

In [7]:
s1s2.head(10)

,date,h,m,foF2BR52P,foF2NI63_,foF2TV51R,foF2_avg_s1,foF2_n_sondes_s1,foF2DW41K,foF2KJ609,...,foF2VA50L,foF2CN53L,foF2PY52R,foF2GH64L,foF2HO54K,foF2CB53N,foF2_avg_s2,foF2_n_sondes_s2,lbl,deviation
0,2004-11-03,0,0,7.988000,8.39100,8.917667,8.432222,3,NaN,NaN,...,NaN,7.99745,NaN,7.35800,8.34300,NaN,7.899483,3,not_eq,0.532739
1,2004-11-03,0,15,8.106375,8.65200,9.160583,8.639653,3,NaN,NaN,...,NaN,8.09520,NaN,7.43300,8.24850,NaN,7.925567,3,not_eq,0.714086
2,2004-11-03,0,30,8.224750,8.91300,9.403500,8.847083,3,NaN,NaN,...,NaN,8.19295,NaN,7.50800,8.15400,NaN,7.951650,3,not_eq,0.895433
3,2004-11-03,0,45,8.343125,9.17400,9.646417,9.054514,3,NaN,NaN,...,NaN,8.29070,NaN,7.58300,8.05950,NaN,7.977733,3,not_eq,1.076781
4,2004-11-03,1,0,8.461500,9.43500,9.889333,9.261944,3,NaN,NaN,...,NaN,8.38845,NaN,7.65800,7.96500,NaN,8.003817,3,not_eq,1.258128
5,2004-11-03,1,15,8.579875,11.53225,10.132250,10.081458,3,NaN,NaN,...,NaN,8.49720,NaN,7.61675,8.10725,NaN,8.073733,3,not_eq,2.007725
6,2004-11-03,1,30,8.698250,13.62950,10.375167,10.900972,3,NaN,NaN,...,NaN,8.60870,NaN,7.57550,8.24950,NaN,8.144567,3,not_eq,2.756406
7,2004-11-03,1,45,8.816625,15.72675,10.618083,11.720486,3,NaN,NaN,...,NaN,8.72020,NaN,7.53425,8.39175,NaN,8.215400,3,not_eq,3.505086
8,2004-11-03,2,0,8.935000,17.82400,10.861000,12.540000,3,NaN,NaN,...,NaN,8.83170,NaN,7.49300,8.53400,NaN,8.286233,3,not_eq,4.253767
9,2004-11-03,2,15,9.093000,15.67850,10.848750,11.873417,3,NaN,NaN,...,NaN,8.90000,NaN,7.46375,8.49900,NaN,8.287583,3,not_eq,3.585833
